<a href="https://colab.research.google.com/github/athospugliesedev/python_codes/blob/main/nttdata2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd

df = pd.read_csv("data.csv")

In [21]:
df.head()

,ano,id_municipio,area,desmatado,incremento,floresta,nuvem,nao_observado,nao_floresta,hidrografia
0,2000,1100015,7137,1761.1,NaN,3639.6,0.0,6.1,1708.0,22.2
1,2000,1100023,4481,2661.2,NaN,1730.0,0.0,0.0,89.6,0.2
2,2000,1100031,1322,851.2,NaN,455.2,0.0,0.6,11.3,3.7
3,2000,1100049,3821,2311.3,NaN,1477.7,0.0,0.0,26.2,5.8
4,2000,1100056,2805,731.3,NaN,829.3,0.0,0.0,1244.4,0.0


In [22]:
df.iloc[0].to_frame()

,0
ano,2000.0
id_municipio,1100015.0
area,7137.0
desmatado,1761.1
incremento,NaN
floresta,3639.6
nuvem,0.0
nao_observado,6.1
nao_floresta,1708.0
hidrografia,22.2


In [23]:
df["id_municipio"] = df["id_municipio"].astype(str)
df["area"] = df["area"].astype(str)
df["desmatado"] = df["desmatado"].astype(str)
df["prompt_text"] = df["id_municipio"] + " de area " + df["area"] + " foi desmatado em: " + df["desmatado"]

In [24]:
df.head()

,ano,id_municipio,area,desmatado,incremento,floresta,nuvem,nao_observado,nao_floresta,hidrografia,prompt_text
0,2000,1100015,7137,1761.1,NaN,3639.6,0.0,6.1,1708.0,22.2,1100015 de area 7137 foi desmatado em: 1761.1
1,2000,1100023,4481,2661.2,NaN,1730.0,0.0,0.0,89.6,0.2,1100023 de area 4481 foi desmatado em: 2661.2
2,2000,1100031,1322,851.2,NaN,455.2,0.0,0.6,11.3,3.7,1100031 de area 1322 foi desmatado em: 851.2
3,2000,1100049,3821,2311.3,NaN,1477.7,0.0,0.0,26.2,5.8,1100049 de area 3821 foi desmatado em: 2311.3
4,2000,1100056,2805,731.3,NaN,829.3,0.0,0.0,1244.4,0.0,1100056 de area 2805 foi desmatado em: 731.3


In [25]:
!pip install python-dotenv

In [26]:
!pip install openai

In [27]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"]
os.environ['OPENAI_API_BASE']

'https://openai-portodigital.openai.azure.com/'

In [28]:
import openai

openai.api_type = "azure"
openai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_version = "2023-05-15"

response = openai.Embedding.create(
    input="OI BRASIL",
    engine="text-similarity-ada-001"
)

In [29]:
embeddings = response['data'][0]['embedding']
embeddings

[-0.019395481795072556,
 0.030103718861937523,
 -0.0001273331290576607,
 0.032396186143159866,
 0.02742917463183403,
 -0.0012279280927032232,
 0.02847486175596714,
 -0.013141470029950142,
 0.050112538039684296,
 -0.026041626930236816,
 0.00418526167050004,
 -0.026262830942869186,
 0.03752407431602478,
 -0.019174279645085335,
 -0.008516316302120686,
 -0.031229844316840172,
 -0.021336035802960396,
 -0.007430410478264093,
 -0.004117392469197512,
 -0.01598694548010826,
 0.043154697865247726,
 -0.013915680348873138,
 0.028434641659259796,
 -0.00904921442270279,
 -0.020421059802174568,
 0.018420178443193436,
 -0.031873345375061035,
 0.0047282143495976925,
 -0.01569535955786705,
 -0.01875198259949684,
 -0.04987122491002083,
 0.015946727246046066,
 0.0006541828042827547,
 0.06603915244340897,
 0.03975621610879898,
 -0.02026018500328064,
 -0.04027905687689781,
 -0.011854470707476139,
 0.011502556502819061,
 0.01390562579035759,
 -0.045608039945364,
 -0.00938604585826397,
 0.001997111365199089,


In [30]:
import time

def text_embedding(text):
    """
    Recebe string, faz chamada ao modelo de embeddings da openai e retorna os embeddings da string
    """
    response = None
    retry_count = 0

    while response is None:
        try:
            response = openai.Embedding.create(engine="text-similarity-ada-001", input=text)
        except Exception as e:
            # Verifique se o erro é devido ao limite de chamadas
            if "call rate limit" in str(e):
                # Espere por um tempo exponencialmente crescente antes de tentar novamente
                time.sleep(2 ** retry_count)
                retry_count += 1
                continue
            else:
                # Se ocorrer outro tipo de erro, lide com ele de acordo com sua lógica de tratamento de erros
                print("Erro ao fazer chamada para a API: ", str(e))
                return None

    return response["data"][0]["embedding"]

In [31]:
# Cria sample das linhas para fazer menos chamadas ao modelo
df_menor = df.sample(1000)

In [32]:
df_menor.shape

(1000, 11)

In [33]:
# criar coluna com os embeddings
df_menor["embeddings"] = df_menor["prompt_text"].apply(lambda linha: text_embedding(linha))

In [34]:
df_menor = df_menor.dropna(subset="embeddings", axis=0)

In [35]:
from scipy import spatial

def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 10
) -> tuple[list[str], list[float]]:
    """
    Retorna as linhas do dataframe mais semelhantes a string de busca
    """

    query_embedding = text_embedding(query)

    strings_and_relatednesses = [ ]
    for i, row in df.iterrows():

        #relatedness = relatedness_fn(np.array(query_embedding).flatten(), np.array(row["embedding"]).flatten())
        relatedness = relatedness_fn(query_embedding, row["embeddings"])
        item = (row["prompt_text"], relatedness)
        strings_and_relatednesses.append(item)

    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [48]:
strings, relatednesses = strings_ranked_by_relatedness("Existe alguma area que o valor de desmatado passou de 3000? Só responda areas desmatadas acima de 3000, se o valor de desmatado não for entre 3000 e 4000, oculte-o",
                              df_menor)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.861


'1508407 de area 3793 foi desmatado em: 3256.8'

relatedness=0.858


'1508308 de area 4971 foi desmatado em: 3236.3'

relatedness=0.857


...

'1714203 de area 3260 foi desmatado em: 0.0'

relatedness=0.854


'1508407 de area 3793 foi desmatado em: 3377.7'

relatedness=0.853


'1100130 de area 8590 foi desmatado em: 3369.1'

In [37]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [49]:
import tiktoken

def num_tokens(text: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-16k")
    return len(encoding.encode(text))

In [50]:
num_tokens("oi brasil")

3

In [51]:
num_tokens("Existe alguma area desmatada que passou de 3000?")

16

In [52]:
def montar_query(query: str, df: pd.DataFrame, token_budget: int) -> str:
    linhas, relatednesses = strings_ranked_by_relatedness(query, df)
    introducao = """
    Use o conteúdo abaixo de dados sobre o desmatamento para responder a perguntar a seguir.
    O conteúdo dos dados deve ser a sua única fonte para dar as respostas.
    Se você não encontrar a resposta esperada, escrava "Não consegui identificar uma resposta."
    """
    pergunta = f"\n\Pergunta: {query}"
    messagem = introducao
    for linha in linhas:
        proxima_linha = f'\n\nSeção com dados dos desmatamentos acima de 3000:\n"""\n{linha}\n"""'
        if num_tokens(messagem + proxima_linha + pergunta) > token_budget:
            break
        else:
            messagem = messagem + proxima_linha

    return messagem + pergunta


In [53]:
from pprint import pprint

prompt = montar_query(query="Existe alguma area desmatada que passou de 3000?",
             df=df_menor,
             token_budget=16000)
pprint(prompt)

('\n'
 '    Use o conteúdo abaixo de dados sobre o desmatamento para responder a '
 'perguntar a seguir.\n'
 '    O conteúdo dos dados deve ser a sua única fonte para dar as respostas.\n'
 '    Se você não encontrar a resposta esperada, escrava "Não consegui '
 'identificar uma resposta."\n'
 '    \n'
 '\n'
 'Seção com dados dos desmatamentos acima de 3000:\n'
 '"""\n'
 '1508407 de area 3793 foi desmatado em: 3256.8\n'
 '"""\n'
 '\n'
 'Seção com dados dos desmatamentos acima de 3000:\n'
 '"""\n'
 '1703008 de area 1796 foi desmatado em: 3.5\n'
 '"""\n'
 '\n'
 'Seção com dados dos desmatamentos acima de 3000:\n'
 '"""\n'
 '1703008 de area 1796 foi desmatado em: 3.5\n'
 '"""\n'
 '\n'
 'Seção com dados dos desmatamentos acima de 3000:\n'
 '"""\n'
 '1508407 de area 3793 foi desmatado em: 3339.9\n'
 '"""\n'
 '\n'
 'Seção com dados dos desmatamentos acima de 3000:\n'
 '"""\n'
 '5101902 de area 15999 foi desmatado em: 3005.2\n'
 '"""\n'
 '\n'
 'Seção com dados dos desmatamentos acima de 3000:\

In [54]:
def perguntar(
    query: str,
    df: pd.DataFrame = df,
    model: str = "gpt-35-turbo-16k",
    print_message: bool = False,
) -> str:
    message = montar_query(query, df, token_budget=16000)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Você responde perguntas sobre o desmatamento."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
        engine="gpt-35-turbo-16k"
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [55]:
resposta = perguntar("Existe algum dados que amostra o desmatamento por area abaixo de 3000?",
    df_menor)
print(resposta)

Não, todos os dados fornecidos mostram o desmatamento por área acima de 3000. Não há informações sobre o desmatamento por área abaixo desse valor.
